<center><h1 style="color:green">Text Classification Using FastText</center>

In [1]:
import pandas as pd

df= pd.read_csv("ecommerce_dataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head()

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [2]:
df.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

<b>Drop NA values

In [3]:
df.isna().sum()

category       0
description    1
dtype: int64

In [4]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [5]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [6]:
df['category'] = df['category'].replace("Clothing & Accessories", "Clothing_Accessories")

In [7]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

When we train a fasttext model, it expects labels to be specified with label prefix. We will just create a third column in the dataframe that has label as well as the product description

In [8]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [9]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


<b>Pre-procesing

In [10]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r'[^\w\s\']',' ', text)
text = re.sub(' +', ' ', text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [11]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [12]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


<b>Train Test Split

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [14]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [15]:
train.to_csv("ecommerce_train.txt", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce_test.txt", columns=["category_description"], index=False, header=False)

<b>Train the model and evaluate performance

In [16]:
import fasttext

model = fasttext.train_supervised(input="ecommerce_train.txt")
model.test("ecommerce_test.txt")

(10085, 0.9700545364402579, 0.9700545364402579)

First parameter (10084) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good

<b>Some Predictions

In [17]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.98521435]))

In [18]:
model.predict("Rich Dad Poor Dad")

(('__label__books',), array([1.00001001]))

In [19]:
model.predict("It is a soft shirt")

(('__label__clothing_accessories',), array([0.98929983]))

In [20]:
model.predict("The kitchen was neat and clean")

(('__label__household',), array([1.00000548]))

In [21]:
model.get_nearest_neighbors("painting")

[(0.9971951246261597, 'temperature'),
 (0.9968993067741394, 'vacuum'),
 (0.9965052008628845, 'steam'),
 (0.996281623840332, 'furniture'),
 (0.9961795210838318, 'lid'),
 (0.9960842132568359, 'guard'),
 (0.9956914782524109, 'lamp'),
 (0.9956258535385132, 'grill'),
 (0.9955993294715881, 'hangers'),
 (0.9953635334968567, 'alarm')]